# Create & search embeddings from your journal

1. Upload and
2. Preprocess data
3. Create embeddings from chunked data
4. Search and chat with your journal


- Note: Search is only with OpenAI, not LangChain (as in Pinecone file) or something else

In [43]:
import os
import numpy as np
import pandas as pd
import re
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Pinecone
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import OpenAI
import openai
from openai.embeddings_utils import get_embedding, cosine_similarity

### Upload

Either copy and paste journal entries below or upload a .txt file.

! Text format is important. Format it in the way below,
    (YYYY.MM.DD \n then text of entry)
    
**For upload: change var "text" in line 35 to "file"**
    
 

In [44]:
# short version of test journal data
text = f"""
2023.04.13
Heute ist ein anstrengender Tag. Ich fühle mich seit ein paar Tagen sehr gestresst und kann kaum schlafen. In der Nacht wache ich ständig auf und meine Gedanken kreisen ununterbrochen um all die Dinge, die ich noch erledigen muss. Ich habe das Gefühl, dass ich keine Energie mehr habe und bin deshalb heute den ganzen Tag müde und unkonzentriert. 
Ich hoffe, dass sich das bald ändert und ich wieder zu meiner alten Energie zurückfinde.

2023.04.14
Heute geht es mir schon etwas besser. Ich habe gestern Abend ein paar Entspannungsübungen gemacht und konnte dadurch besser schlafen. Meine Gedanken sind immer noch etwas chaotisch, aber ich habe das Gefühl, dass ich langsam wieder Kontrolle darüber bekomme. 
Ich habe heute auch schon ein paar Dinge von meiner To-Do-Liste abhaken können, was mir ein gutes Gefühl gibt.

2023.04.15
Ich bin wirklich stolz auf mich, denn ich habe heute schon sehr viel geschafft. Ich fühle mich energiegeladen und produktiv. Die Entspannungsübungen scheinen zu helfen und ich kann meine Gedanken besser sortieren. Ich habe sogar schon anfangen können, an einem neuen Projekt zu arbeiten, auf das ich mich schon seit Wochen freue. Es fühlt sich gut an, wieder in die richtige Richtung zu gehen.

2023.04.16
Ich bin so froh, dass ich die letzten Tage so viel Energie hatte. Es hat mir geholfen, die Dinge, die ich schon lange vor mir hergeschoben habe, endlich anzugehen. Heute habe ich fast alles von meiner To-Do-Liste erledigt und fühle mich unglaublich zufrieden. Ich habe das Gefühl, dass ich meine alte Kraft zurückgewonnen habe und bin optimistisch für die Zukunft.

2023.04.17
Heute ist ein guter Tag. Ich fühle mich ausgeglichen und glücklich. Die letzten Tage haben mir gezeigt, dass ich auch in schwierigen Situationen durchhalten kann. 
Ich habe gelernt, dass es wichtig ist, auf mich selbst zu achten und mir Zeit für Entspannung und Regeneration zu nehmen. Ich bin dankbar für alles, was ich erreicht habe und freue mich auf das, was noch kommt.
"""

In [45]:
with open('journals/Journal-short.txt', 'r', encoding="utf-8") as f:
    file = f.read()

In [46]:
# define openai api key and embedding model
openai.api_key = os.getenv('OPENAI_API_KEY')
API_0 = os.getenv('OPENAI_API_KEY')
EMBEDDING_MODEL = "text-embedding-ada-002"
GPT_MODEL = "gpt-3.5-turbo"
path_embeddings = "embeddings"

In [47]:
# LangChain: initialize OpenAI Embeddings Model + create embeddings
embeddings = OpenAIEmbeddings(model_name="ada")

### Preprocessing the text

1. Split entries and 
2. Chunk entries into smaller pieces

    The result depends heavily on the setting of the chunking size and overlap. 
    Change length for different results!

In [48]:
def split_text(content):
    """Split content into entries based on date pattern and return a Pandas DataFrame.
       content: string with journal entries
       df: Pandas DataFrame with dates and entries
       
       WORKS ONLY WITH YYYY.MM.DD FORMAT"""
    # Define a regular expression pattern to match dates
    date_pattern = r'\d{4}.\d{2}.\d{2}'

    # Split the content based on the date pattern
    entries = re.split(date_pattern, content)

    # Extract dates from the content
    dates = re.findall(date_pattern, content)

    # Create a dictionary with dates and corresponding entries
    data = {'Date': dates, 'Text': [entry.strip() for entry in entries[1:]]}
 
    # Create a Pandas DataFrame from the dictionary and return it
    return pd.DataFrame(data)

In [82]:
# function to split entries into chunks using langchain
def langchain_textsplitter(content_entry):
    
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size = 100, #change
        chunk_overlap = 20, #change
    )
    
    text_array = []
    texts = text_splitter.create_documents([content_entry])
    for chunk in texts:
        text_array.append(chunk.page_content)

    return text_array

In [50]:
df = split_text(text)
df.head()

,Date,Text
0,2023.04.13,Heute ist ein anstrengender Tag. Ich fühle mic...
1,2023.04.14,Heute geht es mir schon etwas besser. Ich habe...
2,2023.04.15,"Ich bin wirklich stolz auf mich, denn ich habe..."
3,2023.04.16,"Ich bin so froh, dass ich die letzten Tage so ..."
4,2023.04.17,Heute ist ein guter Tag. Ich fühle mich ausgeg...


In [51]:
chunked_df = pd.DataFrame(columns=['Date', 'Text'])

for index, row in df.iterrows():
    # split text into chunks, return: list of strings
    chunks = langchain_textsplitter(row['Text'])
    date_vector = [row['Date']]*len(chunks)
    # concatenate in new dataframe
    chunked_df = pd.concat([chunked_df, pd.DataFrame({'Date': date_vector, 'Text': chunks})], ignore_index = True)

chunked_df.head()

,Date,Text
0,2023.04.13,Heute ist ein anstrengender Tag. Ich fühle mic...
1,2023.04.13,und kann kaum schlafen. In der Nacht wache ich...
2,2023.04.13,"ununterbrochen um all die Dinge, die ich noch ..."
3,2023.04.13,dass ich keine Energie mehr habe und bin desha...
4,2023.04.13,"Ich hoffe, dass sich das bald ändert und ich w..."


### Create Embeddings & store them

Now create embeddings from the chunks.
**For large datasets it can take a while!**

Therefore generate once and save it as CSV-file.

TODO:
- compare results from Langchain, OpenAI and Open Source models similar to Word2Vec or Sent2Vec

In [52]:
chunked_df['embedding'] = chunked_df['Text'].apply(lambda x: get_embedding(x, engine=EMBEDDING_MODEL))
chunked_df.head()

,Date,Text,embedding
0,2023.04.13,Heute ist ein anstrengender Tag. Ich fühle mic...,"[-0.013216842897236347, 0.021550197154283524, ..."
1,2023.04.13,und kann kaum schlafen. In der Nacht wache ich...,"[0.003639327362179756, -0.0006552351405844092,..."
2,2023.04.13,"ununterbrochen um all die Dinge, die ich noch ...","[-0.02012534998357296, -0.016028128564357758, ..."
3,2023.04.13,dass ich keine Energie mehr habe und bin desha...,"[-0.015418408438563347, 0.0031841034069657326,..."
4,2023.04.13,"Ich hoffe, dass sich das bald ändert und ich w...","[2.216765278717503e-05, -0.017116189002990723,..."


In [24]:
# save as .csv to use it later, if wanted
chunked_df.to_csv('journal_embedding.csv')

# Chat & Search

1. Set the hyperparamters according to your needs and questions.
2. Use OpenAIs GPT to generate an readable answer from search result.

! Language ! 
\\\
Side note: If the journal is in german, the search term should also be. Otherwise the result of cosine similarity could get worse

In [56]:
# function to search in embeddings
def search(df, search_term, n=3):
    """ 
    df: dataframe with embeddings
    search_term: string to search for
    n: number of results to return
    """
    # convert embeddings to numpy array
    #df["embedding"] = df["embedding"].apply(eval).apply(np.array)

    # get embedding of search term
    search_embeddings = get_embedding(search_term, engine=EMBEDDING_MODEL)

    # calculate cosine similarity
    df["similarity"] = df["embedding"].apply(lambda x: cosine_similarity(x, search_embeddings))

    # sort by similarity and return top n
    return df.sort_values("similarity", ascending=False).head(n)

Define OpenAI call-function:

In [74]:
def get_completion(prompt, model="gpt-4", temperature=0): # gpt-3.5-turbo
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model = model,
        messages = messages,
        temperature = temperature,
    )
    return response.choices[0].message["content"]

In [83]:
# todo: try different models, from "gpt-3.5-turbo", anthropics "claude" to open source models on hugging face.

### Hyperparameter setting: 
- How much similar vectors you want gpt to compare for an answer?
- What is your search-term?

In [81]:
num_most_similar = 5
query = "Why is the author feeling awesome?" # change
most_similar = search(chunked_df, query, num_most_similar)
most_similar 
# outputs the most relevant parts of your journal. 
# can be fine-tuned by change the chunking length

,Date,Text,embedding,similarity
9,2023.04.14,was mir ein gutes Gefühl gibt.,"[-0.005818309728056192, -0.017997801303863525,...",0.785841
14,2023.04.15,"freue. Es fühlt sich gut an, wieder in die ric...","[0.02654009684920311, -0.01804625801742077, -0...",0.777955
18,2023.04.16,"zufrieden. Ich habe das Gefühl, dass ich meine...","[-0.00176370854023844, -0.027162054553627968, ...",0.768509
10,2023.04.15,"Ich bin wirklich stolz auf mich, denn ich habe...","[-0.004994497634470463, -0.0016195483040064573...",0.759255
24,2023.04.17,"erreicht habe und freue mich auf das, was noch...","[0.008538960479199886, -0.03639346361160278, -...",0.749236


### Result:

- specifying the prompt
- create answer out of combined outputs from most relevant search results (cos-sim)

Todo: manually try different similarity calculation methods. does the result change?

In [87]:
conc_text = '\n\n'.join(most_similar["Text"].tolist())
prompt = f"""
Take the users query and answer it by including what the user has written
in his journal.
query: '{query}'
journal parts: '{conc_text}'

"""
print(f'Your question was: {query} \n\nThe anwer according to your journal is: \n{get_completion(prompt)}')

Your question was: Why is the author feeling awesome? 

The anwer according to your journal is: 
The author is feeling awesome because he is making progress and moving in the right direction. He feels like he has regained his old strength and is optimistic about the future. He is proud of what he has achieved so far and is looking forward to what is to come.


In [68]:
# without loading data every time: change search function, maybe create copy of df and convert this. Otherwise: Error trying converting df['embeddings'] again

In [88]:
# same process, different prompt
prompt2 = f"""You are in the position of a therapist.
            The patient asks you a question. You also get important parts of the journal of the user. Answer this question based on the information from parts of the journal given below.
            Question: {query}
            Important parts of journal: {conc_text}

            Goals:
            1. Help people to overcome the things which are holding them back.
            2. Discover recurring patterns.
            Answer:


            Example
            Question: Why was I feeling so good?
            Important parts of journal: I've meditated a lot. Read a lot.
            Answer: Because meditation and reading were core habits in your life.
            """

In [89]:
print(f'Your question was: {query} \n\nThe anwer according to your journal is: \n{get_completion(prompt2)}')

Your question was: Why is the author feeling awesome? 

The anwer according to your journal is: 
The author is feeling awesome because they are making progress and moving in the right direction. They feel they have regained their old strength and are optimistic about the future. They are proud of what they have accomplished so far and are looking forward to what is to come.


In [90]:
# todo: build chatwindow
# ! use from other file and include search